## importing the messwert Table to the Postgres Database

The aim of this Notebook is to import the Messwert Table which is provided by LANUV to our postgres Database.

first we need follwing pachakges:
* Pandas 
    
    Pandas is essentioal to read and manage our data in the data frame.
* sqlalchemy:
 
 sqlalchemy is essentioal to read from database, write to database, connect to database and so on.


In [2]:
import pandas as pd
import sqlalchemy
import credential as  creds 

## read the Data

The name of our CSV file is "opendata_messwert" which is avaliable in data folder. 

In [2]:
data_in_dir = r"../data/"

gw_quality_fname = r"opendata_messwert.csv"

gw_quality_pfname = data_in_dir + gw_quality_fname

Now we need to read our csv files in the pandas data fram
##### Don't forget to add the "delimiter", normal csv file's delimiter is "," but in our case it's ";", so we need to assign it.

Also in the same time, I have assigned the "str" type to the two columns. and because "sl nr" is unique value then I decided to choose it as an index.


In [6]:
df_qual = pd.read_csv(gw_quality_pfname, sep = ";", index_col=["sl_nr"], dtype = {"messergebnis_c":str ,"messergebnis_hinweis":str })
df_qual.head()

C:\Users\sinan\anaconda3\envs\geo\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,messstelle_id,pna_id,datum_pn,stoff_nr,probengut,messergebnis_c,messergebnis_hinweis,bestimmungsgrenze,masseinheit,trennverfahren,verfahren,vor_ort,herkunft,aktual_dat,erstell_dat
sl_nr,,,,,,,,,,,,,,,
2903561,59620687,5/2005/4599,20051018,1164,Grundwasser,22.00000,NaN,NaN,µg/l,Gesamtgehalt,DIN 38406-E22 MAERZ 1988,,HYGC_BR-AR,20051205,20051205
2903564,59620687,5/2005/4599,20051018,1061,Grundwasser,6.80000,NaN,NaN,-,Gesamtgehalt,DIN 38404-C5 JANUAR 1984,ja,HYGC_BR-AR,20051205,20051205
2903565,59620687,5/2005/4599,20051018,1011,Grundwasser,12.80000,NaN,NaN,°C,Gesamtgehalt,DIN 38404-C4 DEZEMBER 1976,ja,HYGC_BR-AR,20051205,20051205
2903584,59620389,5/2005/5002,20051114,1011,Grundwasser,12.30000,NaN,NaN,°C,Gesamtgehalt,DIN 38404-C4 DEZEMBER 1976,ja,HYGC_BR-AR,20051205,20051205
2903585,59620080,5/2005/5001,20051111,1061,Grundwasser,7.40000,NaN,NaN,-,Gesamtgehalt,DIN 38404-C5 JANUAR 1984,ja,HYGC_BR-AR,20051205,20051205


## changing the Data type
we need to change some columns's data type. 

 we have three date columns but they are not in Date type, they are just simply object which means string.

In [4]:
df_qual.dtypes

messstelle_id             int64
pna_id                   object
datum_pn                  int64
stoff_nr                  int64
probengut                object
messergebnis_c           object
messergebnis_hinweis     object
bestimmungsgrenze       float64
masseinheit              object
trennverfahren           object
verfahren                object
vor_ort                  object
herkunft                 object
aktual_dat                int64
erstell_dat               int64
dtype: object

In [15]:
df_qual['datum_pn'] = pd.to_datetime(df_qual['datum_pn'], format = '%Y-%m-%d')

df_qual[ 'aktual_dat'] = pd.to_datetime(df_qual[ 'aktual_dat'], format = '%Y-%m-%d')

df_qual['erstell_dat'] = pd.to_datetime(df_qual['erstell_dat'], format = '%Y-%m-%d')

In [16]:
df_qual.dtypes

messstelle_id                    int64
pna_id                          object
datum_pn                datetime64[ns]
stoff_nr                         int64
probengut                       object
messergebnis_c                  object
messergebnis_hinweis            object
bestimmungsgrenze              float64
masseinheit                     object
trennverfahren                  object
verfahren                       object
vor_ort                         object
herkunft                        object
aktual_dat              datetime64[ns]
erstell_dat             datetime64[ns]
dtype: object

* Now all the tree columns which were date are in datetime64 type

### Cretae functions

Now the most important columns is "messergebnis_c" which simply means the result of the measurments. The data type of this columns should be float. But we have probelm. some value in "messergebnis_c" starts with < or > and that's why the pandas automatically assign object data type to this column.  First we need to create some functions to check that how many "messergebnis_c" consist of < and how many consist of > and how many value of "messergebnis_c" are with no greater than and less than sign. 

* First function: 

 This function is to check that are we able to convert the string to float?

In [17]:
# check if string can be converted to float
def is_float(element: str) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

* Second function: 

The function is to check that the str starts with '<' 

In [18]:
# check if string starts with '<'
def is_less(element: str) -> bool:
    return element[0] == "<" 

* Third function:

The function is to check that the str starts with '>'

In [19]:
# check if string starts with '>'
def is_greater(element: str) -> bool:
    return element[0] == ">" 

### Now apply all the three above functions on the "messergebnis_c" columns

In [20]:
%time idx_mess_is_float   = df_qual["messergebnis_c"].apply(is_float)
#%time idx_mess_not_float  = ~idx_mess_is_float
%time idx_mess_is_less    = df_qual["messergebnis_c"].apply(is_less)
%time idx_mess_is_greater = df_qual["messergebnis_c"].apply(is_greater)

Wall time: 3.62 s
Wall time: 1.15 s
Wall time: 809 ms


### Now we can see that "messergebnis_c" of how many rows are with "<" and ">" or with no sign which means they can simply convert to float with no problem.

In [24]:
# How many can be converted to float with no problem
df_qual[idx_mess_is_float].shape

(1697167, 15)

In [25]:
# How many consist of < 
df_qual[idx_mess_is_less].shape

(1974713, 15)

In [28]:
# how many consist of >
df_qual[idx_mess_is_greater].shape

(33, 15)

## check again

Now is the time to check again all the value of "messergebnis_c" to be sure that there is no other probelm

In [8]:
assert idx_mess_is_float.sum() + (~idx_mess_is_float).sum() == df_qual.shape[0], "mess has either to be float or not float"

In [9]:
# ((idx_mess_is_less | idx_mess_is_greater) == idx_mess_not_float).value_counts()
assert ((idx_mess_is_less | idx_mess_is_greater) != (~idx_mess_is_float)).sum() == 0, "'not float' supposed to come from '<' or '>' but nothing else."

In [10]:
# Print records which are neither less nor greater nor float -> should be empty data frame
assert df_qual[~idx_mess_is_less & ~idx_mess_is_greater & ~idx_mess_is_float].shape[0] == 0
print(df_qual[~idx_mess_is_less & ~idx_mess_is_greater & ~idx_mess_is_float])

Empty DataFrame
Columns: [messstelle_id, pna_id, datum_pn, stoff_nr, probengut, messergebnis_c, messergebnis_hinweis, bestimmungsgrenze, masseinheit, trennverfahren, verfahren, vor_ort, herkunft, aktual_dat, erstell_dat]
Index: []


## Create a new column

In this step, we want to create a new column with the name of "messergebnis_cm". The value in this column is based on "messergebnis_c". There are three diffrent values in this column.

* = : Which means that the value in messergebnis_c has no sign (< or >)
* > : Which means that the value in messergebnis_c starts with sign of >
* < : Which means that the value in messergebnis_c starts with sogn of <

In [30]:
df_qual.loc[idx_mess_is_float, "messergebnis_cm"] = "="
df_qual.loc[idx_mess_is_less, "messergebnis_cm"] = "<"
df_qual.loc[idx_mess_is_greater, "messergebnis_cm"] = ">"

## replace the signs with noting

Now is the time to delete the < and > from the biggining of each cell. but acctually we have these value in another column (messergebnis_cm)

In [31]:
df_qual.messergebnis_c = df_qual.messergebnis_c.replace({'>':''}, regex=True)

In [32]:
df_qual.messergebnis_c = df_qual.messergebnis_c.replace({'<':''}, regex=True)

## check the datafram again

In [33]:
df_qual

,messstelle_id,pna_id,datum_pn,stoff_nr,probengut,messergebnis_c,messergebnis_hinweis,bestimmungsgrenze,masseinheit,trennverfahren,verfahren,vor_ort,herkunft,aktual_dat,erstell_dat,messergebnis_cm
sl_nr,,,,,,,,,,,,,,,,
2903561,59620687,5/2005/4599,1970-01-01 00:00:00.020051018,1164,Grundwasser,22.00000,NaN,NaN,µg/l,Gesamtgehalt,DIN 38406-E22 MAERZ 1988,,HYGC_BR-AR,1970-01-01 00:00:00.020051205,1970-01-01 00:00:00.020051205,=
2903564,59620687,5/2005/4599,1970-01-01 00:00:00.020051018,1061,Grundwasser,6.80000,NaN,NaN,-,Gesamtgehalt,DIN 38404-C5 JANUAR 1984,ja,HYGC_BR-AR,1970-01-01 00:00:00.020051205,1970-01-01 00:00:00.020051205,=
2903565,59620687,5/2005/4599,1970-01-01 00:00:00.020051018,1011,Grundwasser,12.80000,NaN,NaN,°C,Gesamtgehalt,DIN 38404-C4 DEZEMBER 1976,ja,HYGC_BR-AR,1970-01-01 00:00:00.020051205,1970-01-01 00:00:00.020051205,=
2903584,59620389,5/2005/5002,1970-01-01 00:00:00.020051114,1011,Grundwasser,12.30000,NaN,NaN,°C,Gesamtgehalt,DIN 38404-C4 DEZEMBER 1976,ja,HYGC_BR-AR,1970-01-01 00:00:00.020051205,1970-01-01 00:00:00.020051205,=
2903585,59620080,5/2005/5001,1970-01-01 00:00:00.020051111,1061,Grundwasser,7.40000,NaN,NaN,-,Gesamtgehalt,DIN 38404-C5 JANUAR 1984,ja,HYGC_BR-AR,1970-01-01 00:00:00.020051205,1970-01-01 00:00:00.020051205,=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882791,10407406,1/2004/91121,1970-01-01 00:00:00.020041026,1331,Grundwasser,19.20000,NaN,NaN,mg/l,Gesamtgehalt,DIN 38405-D19 FEBRUAR 1988,,HYGC_BR-K,1970-01-01 00:00:00.020051107,1970-01-01 00:00:00.020051107,=
2882792,10407406,1/2004/91121,1970-01-01 00:00:00.020041026,1472,Grundwasser,5.13000,NaN,NaN,mmol/l,Gesamtgehalt,DIN 38409-H7-1-2 MAI 1979,ja,HYGC_BR-K,1970-01-01 00:00:00.020051107,1970-01-01 00:00:00.020051107,=
2882793,10407406,1/2004/91121,1970-01-01 00:00:00.020041026,1477,Grundwasser,0.35000,NaN,NaN,mmol/l,Gesamtgehalt,DIN 38409-H7-2-2 MAI 1979,,HYGC_BR-K,1970-01-01 00:00:00.020051107,1970-01-01 00:00:00.020051107,=


## converting messergebnis_c to float

Finally we are in the stage that we can now convert the messergebnis_c to float

In [37]:
 df_qual.messergebnis_c = df_qual.messergebnis_c.astype(float)




In [38]:
df_qual.dtypes

messstelle_id                    int64
pna_id                          object
datum_pn                datetime64[ns]
stoff_nr                         int64
probengut                       object
messergebnis_c                 float64
messergebnis_hinweis            object
bestimmungsgrenze              float64
masseinheit                     object
trennverfahren                  object
verfahren                       object
vor_ort                         object
herkunft                        object
aktual_dat              datetime64[ns]
erstell_dat             datetime64[ns]
messergebnis_cm                 object
dtype: object

## Eerything is fine now

As you can simply see above that all the columns's type is what we want. 

* messergebnis_c is now in float
* datum_pn is now in datetime64
* aktual_dat is now in datetime64
* erstell_dat is now in datetime64


## once again check externally to be 100% sure

In [41]:
# write on csv to check externally
df_qual.to_csv("test.csv")

## Import to Database

Now is the time to import our datafram to database with no problem. 

First we need to connect  to our database. and then import the datafram. 

In this case, I have imported the data to schema "Openhype" which has been already created in the PGadmin by myself, but you can import to your Schema.

In [5]:
# connect to out credential file

postgresurl = creds.URL


In [6]:
# replace your database link in below
engine = sqlalchemy.create_engine(postgresurl)

In [1]:
%time df_qual.to_sql(con=engine, name="messwert", schema="Openhype", if_exists="replace", index=False)

# Finish !
## Good Luck, You are doing well